# # VRP Variant Solver Comparison (8-Node Example)

# We will compare three methods for solving a profit-maximizing, duration-constrained vehicle routing problem variant on an 8-node graph:
# 1. Deep Reinforcement Learning (DRL) using DQN.
# 2. Mixed-Integer Programming (MIP) for the optimal solution.
# 3. A simple Greedy Heuristic (Best Reward/Time Ratio).

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
from collections import deque, namedtuple
import matplotlib.pyplot as plt
import time
import pulp 
import math
import os

# # Define Constants (Adjusted for 8 Nodes)

In [ ]:
NUM_NODES = 8
ACTION_SPACE_SIZE = NUM_NODES

# Problem Constraints (Keep the same duration limits for consistency)
DURATION_LIMIT = 60.0
DURATION_TOLERANCE = 0.10
MIN_DURATION = DURATION_LIMIT * (1 - DURATION_TOLERANCE)
MAX_DURATION = DURATION_LIMIT * (1 + DURATION_TOLERANCE)
BIG_M_PENALTY = -1e9 # Large negative number for rewards

# Use a fixed seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# ### Generate Sample Data

In [ ]:
print(f"Generating sample data for {NUM_NODES} nodes...")

# Time Matrix: Asymmetric times between 2 and 15 hours
time_matrix = np.random.uniform(low=2.0, high=15.0, size=(NUM_NODES, NUM_NODES))
# Make slightly asymmetric
asymmetry_factor = np.random.uniform(low=0.8, high=1.2, size=(NUM_NODES, NUM_NODES))
time_matrix *= asymmetry_factor
np.fill_diagonal(time_matrix, 0) # Diagonal time is 0

# Reward Matrix: Rewards between 50 and 500
# Let's make reward somewhat inversely related to time (shorter = higher reward density)
# but with significant noise.
reward_matrix = 50 + 450 * np.random.rand(NUM_NODES, NUM_NODES)
# Add some inverse time correlation + noise
time_factor = 1 / (time_matrix + 1e-6) # Add epsilon to avoid division by zero
time_factor_scaled = (time_factor / np.max(time_factor)) * 200 # Scale inverse time effect
noise = np.random.uniform(-50, 50, size=(NUM_NODES, NUM_NODES))
reward_matrix = np.clip(reward_matrix * 0.5 + time_factor_scaled + noise, 50, 500) # Combine & clip
np.fill_diagonal(reward_matrix, 0) # Diagonal reward is 0 (will be penalized later)

# Round for clarity
time_matrix = np.round(time_matrix, 1)
reward_matrix = np.round(reward_matrix, 0)

# Create DataFrames for easy viewing
time_df = pd.DataFrame(time_matrix, index=range(NUM_NODES), columns=range(NUM_NODES))
reward_df = pd.DataFrame(reward_matrix, index=range(NUM_NODES), columns=range(NUM_NODES))

print("\nSample Time Matrix (hours):")
print(time_df)
print("\nSample Reward Matrix:")
print(reward_df)

# Apply Big M penalty to reward matrix diagonal (used by DRL and Heuristic)
reward_matrix_penalized = reward_matrix.copy()
np.fill_diagonal(reward_matrix_penalized, BIG_M_PENALTY)

# **Data Description:**
# * **Nodes:** 8 locations, indexed 0 through 7.
# * **Time Matrix:** Shows the travel time in hours between any two nodes `i` and `j` (`time_matrix[i][j]`). Times are generally between 2 and 15 hours and are slightly asymmetric (travel `i` to `j` might take slightly different time than `j` to `i`). Diagonal is 0.
# * **Reward Matrix:** Shows the reward (profit) gained by traveling between nodes `i` and `j` (`reward_matrix[i][j]`). Rewards range roughly from 50 to 500. There's a loose inverse correlation with time (shorter trips *tend* to have higher reward density) plus random noise. Diagonal is 0.
#
# **Goal Reminder:** For each starting node, find a route (cycle) that starts and ends at that node, maximizes total reward, and has a total duration around 60 hours.

## Part 2: DRL Implementation and Training
### DRL Hyperparameters and Agent Definition (Using PyTorch)

In [ ]:
# DRL Hyperparameters (Can potentially reduce episodes/steps for smaller problem)
STATE_SIZE = 2 # (current_node_index, time_elapsed_normalized)
LEARNING_RATE = 0.001
GAMMA = 0.95 # Discount factor

EPSILON_START = 1.0
EPSILON_END = 0.05 # Can end higher for smaller problems
EPSILON_DECAY_STEPS = 5000 # Decay faster for smaller problems?

BUFFER_SIZE = 10000 # Smaller buffer might be okay
BATCH_SIZE = 32 # Smaller batch size

NUM_EPISODES = 3000 # Reduced episodes for 8 nodes
MAX_STEPS_PER_EPISODE = 50 # Max steps per route attempt
TARGET_UPDATE_FREQ = 50 # Update target net more frequently

# Rewards / Penalties (Keep the same)
RETURN_SUCCESS_BONUS = 100
TIME_VIOLATION_PENALTY = -1000
INCOMPLETE_PENALTY = -200

# PyTorch Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Q-Network Definition (Same as before)
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 64) # Smaller network might suffice
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# DQN Agent Definition (Same DQNAgent_PyTorch class as before)
# Ensure the DQNAgent_PyTorch class definition from the previous response is included here
class DQNAgent_PyTorch:
    def __init__(self, state_size, action_size, learning_rate, gamma, buffer_size, batch_size, device):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.gamma = gamma
        self.batch_size = batch_size
        self.device = device

        # Use the smaller QNetwork
        self.policy_net = QNetwork(state_size, action_size).to(self.device)
        self.target_net = QNetwork(state_size, action_size).to(self.device)
        self.update_target_model()
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.loss_function = nn.MSELoss()
        self.epsilon = EPSILON_START

    def update_target_model(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        if random.random() <= self.epsilon:
            current_node_index = int(state[0])
            possible_actions = list(range(self.action_size))
            if current_node_index in possible_actions:
                 possible_actions.remove(current_node_index)
            if not possible_actions:
                return random.randrange(self.action_size)
            return random.choice(possible_actions)
        else:
            self.policy_net.eval()
            with torch.no_grad():
                q_values = self.policy_net(state_tensor)
            self.policy_net.train()
            q_values_numpy = q_values.cpu().data.numpy()[0]
            current_node_index = int(state[0])
            if 0 <= current_node_index < self.action_size:
                q_values_numpy[current_node_index] = -np.inf
            return np.argmax(q_values_numpy)

    def replay(self):
        if len(self.memory) < self.batch_size:
            return 0.0
        minibatch = random.sample(self.memory, self.batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in minibatch])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in minibatch])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in minibatch])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in minibatch])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in minibatch]).astype(np.uint8)).float().to(self.device)

        with torch.no_grad():
            target_q_next = self.target_net(next_states)
            max_q_next = target_q_next.max(1)[0].unsqueeze(1)
            target_q_values = rewards + (self.gamma * max_q_next * (1 - dones))

        current_q_values = self.policy_net(states)
        action_q_values = current_q_values.gather(1, actions)
        loss = self.loss_function(action_q_values, target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def decay_epsilon(self, current_step):
         self.epsilon = max(EPSILON_END, EPSILON_START - (EPSILON_START - EPSILON_END) * (current_step / EPSILON_DECAY_STEPS))

    def load(self, path):
        try:
             self.policy_net.load_state_dict(torch.load(path, map_location=self.device))
             self.update_target_model()
             print(f"Model weights loaded from {path}")
        except Exception as e:
             print(f"Error loading model weights: {e}")

    def save(self, path):
        try:
             os.makedirs(os.path.dirname(path), exist_ok=True)
             torch.save(self.policy_net.state_dict(), path)
             print(f"Model weights saved to {path}")
        except Exception as e:
             print(f"Error saving model weights: {e}")

### DRL Training Loop

In [ ]:
drl_agent = DQNAgent_PyTorch(state_size=STATE_SIZE,
                           action_size=ACTION_SPACE_SIZE,
                           learning_rate=LEARNING_RATE,
                           gamma=GAMMA,
                           buffer_size=BUFFER_SIZE,
                           batch_size=BATCH_SIZE,
                           device=device)

# Training History
drl_episode_rewards = []
drl_episode_losses = []
drl_total_steps = 0
drl_start_train_time = time.time()

print("Starting DRL Training...")

for episode in range(NUM_EPISODES):
    start_node = random.randint(0, NUM_NODES - 1) # Train on random start nodes
    current_node = start_node
    time_elapsed = 0.0
    state = np.array([current_node, time_elapsed / MAX_DURATION], dtype=np.float32)

    episode_reward = 0
    episode_loss_sum = 0
    steps_in_episode = 0
    done = False

    for step in range(MAX_STEPS_PER_EPISODE):
        action = drl_agent.act(state)
        next_node = action
        step_time = time_matrix[current_node][next_node]
        # Use penalized reward matrix for DRL training decisions (implicitly via learned Q)
        # but store experience based on actual rewards + terminal bonus/penalty
        step_reward = reward_matrix[current_node][next_node] # Base reward for the step

        next_time_elapsed = time_elapsed + step_time
        next_state = np.array([next_node, min(next_time_elapsed, MAX_DURATION) / MAX_DURATION], dtype=np.float32)

        terminal_reward = 0
        done = False

        # Termination checks (Same logic as before)
        if next_node == start_node:
            if MIN_DURATION <= next_time_elapsed <= MAX_DURATION:
                terminal_reward = RETURN_SUCCESS_BONUS
                done = True
            elif next_time_elapsed < MIN_DURATION:
                 terminal_reward = INCOMPLETE_PENALTY
                 done = True
            else: # > MAX_DURATION
                 terminal_reward = TIME_VIOLATION_PENALTY
                 done = True
        elif next_time_elapsed > MAX_DURATION:
            terminal_reward = TIME_VIOLATION_PENALTY
            done = True

        # Total reward for the experience tuple
        total_reward_experience = step_reward + terminal_reward

        drl_agent.remember(state, action, total_reward_experience, next_state, done)

        state = next_state
        current_node = next_node
        time_elapsed = next_time_elapsed
        episode_reward += step_reward # Track sum of actual step rewards
        steps_in_episode += 1
        drl_total_steps += 1

        drl_agent.decay_epsilon(drl_total_steps)
        loss = drl_agent.replay()
        if loss > 0: episode_loss_sum += loss
        if drl_total_steps % TARGET_UPDATE_FREQ == 0: drl_agent.update_target_model()
        if done:
            episode_reward += terminal_reward # Add final bonus/penalty for logging
            break

    drl_episode_rewards.append(episode_reward)
    avg_loss = episode_loss_sum / steps_in_episode if steps_in_episode > 0 else 0
    drl_episode_losses.append(avg_loss)

    if (episode + 1) % (NUM_EPISODES // 10) == 0: # Print progress 10 times
         print(f"DRL Episode: {episode + 1}/{NUM_EPISODES}, Steps: {steps_in_episode}, Total Steps: {drl_total_steps}, Reward: {episode_reward:.0f}, Avg Loss: {avg_loss:.4f}, Epsilon: {drl_agent.epsilon:.3f}")

drl_training_time = time.time() - drl_start_train_time
print(f"\nDRL Training Finished. Total time: {drl_training_time:.2f} seconds")